In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive, files
drive.mount("/content/drive")
drive_path = "/content/drive/My Drive/CSCI-576"

In [ ]:
def read_image(filename):
    img = cv2.imread(f"{drive_path}/{filename}", cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"File not found: {pathname}")
    assert img.dtype == np.uint8
    return img.astype(np.float32) / 255.0, img.shape[0], img.shape[1]

In [ ]:
def write_image(M, pathname):
    if M.ndim != 2:
        raise ValueError("Matrix must be 2-dimensional")
    img = np.clip(M, 0.0, 1.0) * 255.0
    img = img.astype(np.uint8)
    if not cv2.imwrite(pathname, img): #, cv2.IMWRITE_JPEG_QUALITY, 100
        raise IOError(f"Could not write image to {pathname}")

In [ ]:
def display_image(M, title):
    ROWS, COLUMNS = M.shape
    if M.ndim != 2:
        raise ValueError("Matrix must be 2-dimensional")
    img = np.clip(M, 0.0, 1.0)
    plt.figure(figsize=(ROWS / 128, COLUMNS / 128), dpi=128)
    plt.imshow(img, cmap='gray', vmin=0.0, vmax=1.0)
    if title: plt.title(f"{title} {ROWS}x{COLUMNS}")
    else: plt.title(f"{ROWS}x{COLUMNS}")
    plt.axis('off')
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
image_names = [
    "grayMan"
  , "grayMan_gaussian"
  , "grayMan_saltandpepper"
  , "Q4_original"
  , "Q4_corrupted"
]
for name in image_names:
    M, _, _ = read_image(f"{name}.png")
    print(M.shape)

In [ ]:
# Question 1
M, ROWS, COLUMNS = read_image(f"grayMan.png")
R = -1
while R < 0 or R > ROWS:
    try: R = int(input(f"Number of rows up to {ROWS}:"))
    except: pass
C = -1
while C < 0 or C > COLUMNS:
    try: C = int(input(f"Number of columns up to {COLUMNS}: "))
    except: pass
rows = np.random.choice(np.arange(0, ROWS), size=R, replace=False)
columns = np.random.choice(np.arange(0, COLUMNS), size=C, replace=False)
S = M
S = np.delete(S, rows, axis=0)
S = np.delete(S, columns, axis=1)
display_image(M, f"Original")
display_image(S, f"Sliced Up!")
write_image(S, f"{drive_path}/grayMan.jpg")

In [ ]:
# Question 2
def correlation(img, transform):
    padded = np.pad(img, pad_width=1, mode='edge')
    output = np.zeros_like(img)
    for r in range(ROWS):
        for c in range(COLUMNS):
            output[r, c] = transform(padded, r, c)
    return output

In [ ]:
# Question 2
avg_kernel = np.array([[1/9, 1/9, 1/9], [1/9, 1/9, 1/9], [1/9, 1/9, 1/9]])
laplacian_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
names = {"grayMan_gaussian": 'g', "grayMan_saltandpepper": 'sb'}
for name in names:
    original, ROWS, COLUMNS = read_image(f"{name}.png")
    avg_smooth    = correlation(original,   lambda I, r, c: np.sum(I[r:r+3, c:c+3] * avg_kernel))
    avg_laplacian = correlation(avg_smooth, lambda I, r, c: np.sum(I[r:r+3, c:c+3] * laplacian_kernel))
    med_smooth    = correlation(original,   lambda I, r, c: np.median(I[r:r+3, c:c+3]))
    med_laplacian = correlation(med_smooth, lambda I, r, c: np.sum(I[r:r+3, c:c+3] * laplacian_kernel))
    display_image(original,       f"Original {name}")
    display_image(avg_smooth,     f"Avg smoothing {name}")
    display_image(avg_laplacian,  f"Avg smooth+laplacian {name}")
    display_image(med_smooth,     f"Median smoothing {name}")
    display_image(med_laplacian,  f"Median smooth+laplacian {name}")
    write_image(avg_smooth,       f"{drive_path}/fnA{names[name]}.jpg")
    write_image(avg_laplacian,    f"{drive_path}/fnA{names[name]}L.jpg")
    write_image(med_smooth,       f"{drive_path}/fnM{names[name]}.jpg")
    write_image(med_laplacian,    f"{drive_path}/fnM{names[name]}L.jpg")

In [ ]:
# Comparison
#   average smoothing works nicely with gaussian noise, but not at all with impulse noise
#   Laplacian with smoothed gaussian noise brings back detail but also brings back the noise
#   median smoothing also works well with gaussian noise, hard to tell which is better
#   median smoothing works very nicely with impulse noise
#   Laplacian with smoothed impulse noise looks very nice

In [ ]:
# Question 3
def display_histogram(hist):
    bins = np.arange(256)
    plt.figure(figsize=(6, 4))
    plt.bar(bins, hist, width=1.0, color='gray', alpha=0.75)
    plt.title("Histogram")
    plt.xlabel('Intensity')
    plt.ylabel('Occurrences')
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
# Question 3
def histogram_calculate(img):
    hist = [0] * 256
    ROWS, COLUMNS = img.shape
    for r in range(ROWS):
        for c in range(COLUMNS):
            hist[int(img[r, c] * 255)] += 1
    return hist

In [ ]:
# Question 3
def CDF_calculate(hist):
    cdf = [0] * 256
    cdf[0] = hist[0]
    for i in range(1, 256):
        cdf[i] = cdf[i - 1] + hist[i]
    return cdf

In [ ]:
# Question 3
def histogram_equalization(img):
    ROWS, COLUMNS = img.shape
    total = ROWS * COLUMNS
    hist = histogram_calculate(img)
    cdf = CDF_calculate(hist)
    cdf_min = cdf[0]
    for i in range(256):
        cdf[i] = (cdf[i] - cdf_min) / (total - cdf_min)
    out = np.zeros_like(img)
    for i in range(ROWS):
        for j in range(COLUMNS):
            out[i, j] = cdf[int(img[i, j] * 255)]
    return out

In [ ]:
# Question 3
def point_processor(img, transform):
    ROWS, COLUMNS = img.shape
    output = np.zeros_like(img)
    for r in range(ROWS):
        for c in range(COLUMNS):
            output[r, c] = transform(img, r, c)
    return output

In [ ]:
# Question 3(a)
name = "Forest"
POWER = 0.55
original, _, _ = read_image(f"Q3_forest.webp")
output = point_processor(original, lambda I, r, c: I[r, c] ** POWER)
display_image(original, f"{name} Original")
display_image(output, f"{name} Power={POWER}")

In [ ]:
# Question 3(b)
names = ["Mushrooms", "Hawkes Bay"]
filenames = ["Q3_mushrooms.png", "Q3_unequalized_hawkes_bay_nz.jpg"]
for n in range(0, 1): # 0, 2
    original, _, _ = read_image(f"{filenames[n]}")
    output = histogram_equalization(original)
    display_image(original, f"{names[n]} Original")
    display_image(output, f"{names[n]} Histogram EQ")

In [ ]:
# Question 3(c)
names = ["Moon", "Misty Mountains"]
filenames = ["Q3_sphx_glr_plot_scientific_005.png", "Q3_misty_mountains.jpg"]
for n in range(0, 1): # 0, 2
    if n == 0: XSLIDE, ROW_BLOCKS, COLUMN_BLOCKS = 4, 16, 20
    if n == 1: XSLIDE, ROW_BLOCKS, COLUMN_BLOCKS = 4, 16, 24
    original, ROWS, COLUMNS = read_image(f"{filenames[n]}")
    display_image(original, f"{names[n]} Original")
    output = np.zeros_like(original, dtype=np.float32)
    weighting = np.zeros_like(original, dtype=np.float32)
    accumulator = np.zeros_like(original, dtype=np.float32)
    height = ROWS // ROW_BLOCKS
    width = COLUMNS // COLUMN_BLOCKS
    for r in range(ROW_BLOCKS * XSLIDE):
        for c in range(COLUMN_BLOCKS * XSLIDE):
            r_start = ((r * height) // XSLIDE); r_end = r_start + height
            c_start = ((c * width) // XSLIDE);  c_end = c_start + width
            weighting[r_start:r_end, c_start:c_end] += 1.0
            accumulator[r_start:r_end, c_start:c_end] += \
                histogram_equalization(original[r_start:r_end, c_start:c_end])
    output = accumulator / weighting
    display_image(output, f"{names[n]} Localized Histogram EQ")

In [ ]:
# Question 3(d)
name = "Spectrum"
original, ROWS, COLUMNS = read_image(f"Q3_spectrum.png")
output = point_processor(original, lambda I, r, c: 4.0 * np.log(1.0 + I[r, c]))
display_image(original, f"{name} Original")
display_image(output, f"{name} Logarithm")

In [ ]:
# Question 3(e)
name = "Bread Rolls"
original, ROWS, COLUMNS = read_image(f"Q3_bread rolls.png")
MIN, MAX = np.min(original), np.max(original)
output = point_processor(original, lambda I, r, c: (I[r, c] - MIN) / (MAX - MIN))
display_image(original, f"{name} Original")
display_image(output, f"{name} Linear Contrast Stretch")

In [ ]:
from skimage.metrics import structural_similarity as ssim
def compare_images(original, restored):
    if original.shape != restored.shape:
        raise ValueError(f"Mismatched shapes: {original.shape} != {restored.shape}")
    mse = np.mean((original - restored) ** 2)
    if mse == 0: psnr = float('inf')
    else:
        data_range = original.max() - original.min()
        if data_range == 0: data_range = 255.0
        psnr = 20 * np.log10(data_range) - 10 * np.log10(mse)
    ssim_metric, diff_map = ssim(original, restored, full=True, data_range=data_range)
    return ssim_metric # (mse, psnr, ssim_metric, diff_map)

In [ ]:
# Question 4
names = ["Q4"]
filenames = ["Q4_corrupted.png"]
laplacian_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
original, _, _ = read_image(f"Q4_original.png")
corrupted, ROWS, COLUMNS = read_image(f"{filenames[n]}")
med_smooth = correlation(corrupted, lambda I, r, c: np.median(I[r:r+3, c:c+3]))
hist_eq = histogram_equalization(med_smooth)
output = correlation(hist_eq, lambda I, r, c: np.sum(I[r:r+3, c:c+3] * laplacian_kernel))
print(compare_images(original, output))
#display_image(original, f"{names[n]} Original")
#display_image(output, f"{names[n]} Restored")